In [0]:
import gensim
from google.colab import drive
drive.mount('/content/mydrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/mydrive


In [0]:
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords
from numpy import dot
from numpy.linalg import norm
import gensim.models.ldamulticore
import os
from sklearn import model_selection
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Add
from keras.layers import Average
from keras.layers import Reshape
from keras.layers import Dropout
from keras.layers import AveragePooling1D
from keras.layers import Lambda
from keras.layers import TimeDistributed
from keras.utils import to_categorical
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras import regularizers
from keras.backend import mean
from keras.backend import sum

Part 2

In [0]:
path="mydrive/My Drive/labeled_articles/"
#load the files from the drive 
allFiles=[]
#every element of list allFiles contains the same text 3 times from every evaluator
for f in os.listdir(path):
  new=True
  for i in range(len(allFiles)):
    if(allFiles[i][0][:-5]==f[:-5]):
      new=False
      allFiles[i].append(f)
  if(new):
    allFiles.append([f])
X=[]
Y=[]
sentences=[]
for i in range(len(allFiles)):
  sentences=[]
  for j in range(len(allFiles[i])):
    sentences.append([])
    filepath = path+allFiles[i][j]
    with open(filepath) as fp:  
      line = fp.readline()
      while line:
        sentences[j].append(line)
        line = fp.readline()
  #sentences contain 3 lists,every element of those lists contain a sentence from different evaluators
  for k in range(len(sentences[0])):
    #ignore comments
    if(sentences[0][k][0]!='#'):
      #for each sentence we extract the class from the 3 different evaluators
      term1=sentences[0][k][:4]
      term2=sentences[1][k][:4]
      term3=sentences[2][k][:4]
      #if 2 evaluators agree for the class of a sentence we accept it,otherwise we ignore the sentence
      if(term1==term2 or term1==term3):
        X.append(sentences[0][k][4:])
        Y.append(term1)
      elif(term2==term3):
        X.append(sentences[0][k][4:])
        Y.append(term2)
trainData=[]

def readFiles(path,docs):
  count=0
  for f in os.listdir(path):
    filepath=path+f
    with open(filepath) as fp:
      count=count+1
      docs.append(fp.read())
      if(count==100):
        break
  return docs
print(len(X))
X=readFiles("mydrive/My Drive/unlabeled_articles/arxiv_unlabeled/",X)
X=readFiles("mydrive/My Drive/unlabeled_articles/jdm_unlabeled/",X)
X=readFiles("mydrive/My Drive/unlabeled_articles/plos_unlabeled/",X)
print(len(X))
for i in range(len(X)):
  Line=remove_stopwords(X[i])
  trainData.append(gensim.utils.simple_preprocess(Line))

1035
1335


In [0]:
firstModel = gensim.models.Word2Vec(
        trainData,
        size=100,
        window=20,
        min_count=2,
        workers=10)
firstModel.train(trainData, total_examples=len(trainData), epochs=10)

print(len(list(firstModel.wv.vocab)))
secondModel = gensim.models.Word2Vec(
        trainData,
        size=100,
        window=1,
        min_count=2,
        workers=10)
secondModel.train(trainData, total_examples=len(trainData), epochs=10)

print(len(list(firstModel.wv.vocab)))
thirdModel = gensim.models.Word2Vec(
        trainData,
        size=100,
        window=10,
        min_count=2,
        workers=10)
secondModel.train(trainData, total_examples=len(trainData), epochs=10)

print(len(list(secondModel.wv.vocab)))

firstModel.wv.save_word2vec_format("mydrive/My Drive/model_word2vec.kv", binary=False)

8632
8632
8632


In [0]:
print("Model with window 20 \n")
listOfWords=firstModel.wv.index2word
for i in range(10):
  print(listOfWords[i])
  print(firstModel.wv.most_similar(positive=listOfWords[i],topn=5))
  print("\n")
  

  
print("Model with window 1 \n")

listOfWords2=secondModel.wv.index2word
for i in range(10):
  print(listOfWords2[i])
  print(secondModel.wv.most_similar(positive=listOfWords2[i],topn=5))
  print("\n")
  
print("Model with window 10 \n")
listOfWords=firstModel.wv.index2word
for i in range(10):
  print(listOfWords[i])
  print(thirdModel.wv.most_similar(positive=listOfWords[i],topn=5))
  print("\n")

Model with window 20 

citation
[('shown', 0.8754635453224182), ('mitchell', 0.823955774307251), ('lattices', 0.8055145740509033), ('scattering', 0.7792379260063171), ('snakes', 0.7704095244407654)]


symbol
[('conditional', 0.9454339742660522), ('th', 0.9439717531204224), ('sum', 0.9425514936447144), ('let', 0.9387654662132263), ('continuity', 0.9360517263412476)]


number
[('rt', 0.8805966973304749), ('ms', 0.8760619163513184), ('trial', 0.8758112192153931), ('inexperienced', 0.8743512630462646), ('accounted', 0.8671337366104126)]


the
[('it', 0.8811230659484863), ('this', 0.8800369501113892), ('step', 0.8741930723190308), ('kcal', 0.8636102676391602), ('symbolic', 0.8609758615493774)]


in
[('montanari', 0.9340759515762329), ('sdp', 0.9321562051773071), ('optimization', 0.9223915338516235), ('tree', 0.9219111800193787), ('laplacian', 0.9217284917831421)]


model
[('models', 0.855358898639679), ('undersampled', 0.7713569402694702), ('biophysically', 0.7413386702537537), ('descriptor

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


For part 2 of the assignment we wanted to check the impact of different values in the window parametre for our word2vec model. So we created 3 model the first one had a window of size 20, the second one got a window of size 1 and the third one size of 10. We get the 5 more similar word for 10 specific words for every model. Some interesting results are the followings

---
Citation

*   window 1 most common word: tversky(Name of scientist) 

*   Window 10 most common word: making

*  Window 20 most common word: shown 

Learing

*   window 1 most common word: turing

*   Window 10 most common word: algorithms

*  Window 20 most common word: multi 

A solid conclusion that we can draw is that window=1 is not enough. As for window 10 and 20 the word citation had a much better approach with the window=20, on the other hand the word learning got a better aproach with window=10. In general though we could say that higher window is better.





 

Part 3

In [0]:
path="mydrive/My Drive/labeled_articles/"
#load the files from the drive 
allFiles=[]
#every element of list allFiles contains the same text 3 times from every evaluator
for f in os.listdir(path):
  new=True
  for i in range(len(allFiles)):
    if(allFiles[i][0][:-5]==f[:-5]):
      new=False
      allFiles[i].append(f)
  if(new):
    allFiles.append([f])
X=[]
Y=[]
sentences=[]
for i in range(len(allFiles)):
  sentences=[]
  for j in range(len(allFiles[i])):
    sentences.append([])
    filepath = path+allFiles[i][j]
    with open(filepath) as fp:  
      line = fp.readline()
      while line:
        sentences[j].append(line)
        line = fp.readline()
  #sentences contain 3 lists,every element of those lists contain a sentence from different evaluators
  for k in range(len(sentences[0])):
    #ignore comments
    if(sentences[0][k][0]!='#'):
      #for each sentence we extract the class from the 3 different evaluators
      term1=sentences[0][k][:4]
      term2=sentences[1][k][:4]
      term3=sentences[2][k][:4]
      #if 2 evaluators agree for the class of a sentence we accept it,otherwise we ignore the sentence
      if(term1==term2 or term1==term3):
        X.append(sentences[0][k][4:])
        Y.append(term1)
      elif(term2==term3):
        X.append(sentences[0][k][4:])
        Y.append(term2)
X_preprossed=[]
for sen in X:
  sen=gensim.utils.simple_preprocess(remove_stopwords(sen))
  fullSen=''
  for i in range(len(sen)):
    fullSen=fullSen+' ' + sen[i]
  X_preprossed.append(fullSen)
  
  
x_train, x_test, y_train, y_test = model_selection.train_test_split(X_preprossed, Y, random_state = 22,stratify=Y)

dictLabel={'AIMX':0,'OWNX':1,'CONT':4,'BASE':3,'MISC':2}

Ytrain=[dictLabel[label] for label in y_train]

# create the tokenizer
tokenizerTrain = Tokenizer()
# fit the tokenizer on the documents
tokenizerTrain.fit_on_texts(x_train)

max_length = max([len(s.split()) for s in x_train])

encoded_docsTrain = tokenizerTrain.texts_to_sequences(x_train)
# pad sequences
Xtrain = pad_sequences(encoded_docsTrain, maxlen=max_length, padding='post')
#print(Xtrain)
vocab_size = len(tokenizerTrain.word_index) + 1

Ytest=[dictLabel[label] for label in y_test]


# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(x_test)

encoded_docs = tokenizer.texts_to_sequences(x_test)
# pad sequences
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(Xtrain)

In [0]:
from numpy import argmax
from numpy import sum as sumarray
def calculateMetrics(predictions,reals,classesLength):
  falsePositives = [0] * classesLength
  truePositives = [0] * classesLength
  falseNegatives = [0] * classesLength
  for i in range(len(reals)):
    myPred = argmax(predictions[i])
    real = reals[i]
    if myPred==real:
      truePositives[real] += 1
    else:
      falsePositives[myPred]+=1
      falseNegatives[real]+=1
  precision=(sumarray(truePositives)/(sumarray(truePositives)+sumarray(falsePositives)))/classesLength
  recall = (sumarray(truePositives)/(sumarray(truePositives)+sumarray(falseNegatives)))/classesLength
  f1 = 2*precision*recall/(precision+recall)
  return precision,recall,f1

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Lambda(lambda x: sum(x, axis=1)))
model.add(Dense(5, activation='sigmoid'))
print(model.summary())


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
predicts=model.predict(Xtest)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
precision,recall,f1 = calculateMetrics(predicts,Ytest,len(dictLabel))
print("Precision = ",precision,"\n Recall = ",recall,"\nF1 = ",f1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_58 (Embedding)     (None, 66, 100)           332300    
_________________________________________________________________
lambda_57 (Lambda)           (None, 100)               0         
_________________________________________________________________
dense_61 (Dense)             (None, 5)                 505       
Total params: 332,805
Trainable params: 332,805
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
 - 3s - loss: 1.3610 - acc: 0.5052
Epoch 2/10
 - 0s - loss: 1.0193 - acc: 0.6418
Epoch 3/10
 - 0s - loss: 0.9509 - acc: 0.6057
Epoch 4/10
 - 0s - loss: 0.8400 - acc: 0.6044
Epoch 5/10
 - 0s - loss: 0.7016 - acc: 0.6044
Epoch 6/10
 - 0s - loss: 0.5649 - acc: 0.6057
Epoch 7/10
 - 0s - loss: 0.4985 - acc: 0.6070
Epoch 8/10
 - 0s - loss: 0.4520 - acc: 0.6108
Epoch 9/10
 - 0s - loss: 0.

From  the results of this model we understand we overfit the model we need to add new layers to stop it

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Dropout(0.9))
model.add(Lambda(lambda x: sum(x, axis=1)))
model.add(Dropout(0.9))
model.add(Dense(5, activation='sigmoid'))
print(model.summary())
# compile network


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
predicts=model.predict(Xtest)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
precision,recall,f1 = calculateMetrics(predicts,Ytest,len(dictLabel))
print("Precision = ",precision,"\n Recall = ",recall,"\nF1 = ",f1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_59 (Embedding)     (None, 66, 100)           332300    
_________________________________________________________________
dropout_17 (Dropout)         (None, 66, 100)           0         
_________________________________________________________________
lambda_58 (Lambda)           (None, 100)               0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_62 (Dense)             (None, 5)                 505       
Total params: 332,805
Trainable params: 332,805
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
 - 3s - loss: 2.5981 - acc: 0.3582
Epoch 2/10
 - 0s - loss: 1.7326 - acc: 0.4291
Epoch 3/10
 - 0s - loss: 1.5958 - acc: 

Adding a Dropout layers with dropout ration 0.7 we prevent overfitting for 10 epochs and we get the expecting results

In [0]:


print(vocab_size,max_length)
print(Xtrain.shape)

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Lambda(lambda x: sum(x, axis=1)))
model.add(Dropout(0.75))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.75))
model.add(Dense(5, activation='softmax'))
print(model.summary())

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
predicts=model.predict(Xtest)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
precision,recall,f1 = calculateMetrics(predicts,Ytest,len(dictLabel))
print("Precision = ",precision,"\n Recall = ",recall,"\nF1 = ",f1)

3323 66
(776, 66)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (None, 66, 100)           332300    
_________________________________________________________________
lambda_29 (Lambda)           (None, 100)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 16)                1616      
_________________________________________________________________
dropout_8 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 5)                 85        
Total params: 334,001
Trainable params: 334,001
Non-trainable params: 0
____________________________________________________

Adding one more dence layer doesn't improve the results and have a toll in execution time and more dropout layers are needed to prevent overfitting

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Lambda(lambda x: mean(x, axis=1)))
model.add(Dense(5, activation='sigmoid'))
print(model.summary())


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
predicts=model.predict(Xtest)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
precision,recall,f1 = calculateMetrics(predicts,Ytest,len(dictLabel))
print("Precision = ",precision,"\n Recall = ",recall,"\nF1 = ",f1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_31 (Embedding)     (None, 66, 100)           332300    
_________________________________________________________________
lambda_31 (Lambda)           (None, 100)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 5)                 505       
Total params: 332,805
Trainable params: 332,805
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
 - 2s - loss: 1.5607 - acc: 0.6044
Epoch 2/10
 - 0s - loss: 1.4607 - acc: 0.6044
Epoch 3/10
 - 0s - loss: 1.3401 - acc: 0.6044
Epoch 4/10
 - 0s - loss: 1.2093 - acc: 0.6044
Epoch 5/10
 - 0s - loss: 1.1067 - acc: 0.6044
Epoch 6/10
 - 0s - loss: 1.0482 - acc: 0.6044
Epoch 7/10
 - 0s - loss: 1.0196 - acc: 0.6044
Epoch 8/10
 - 0s - loss: 1.0046 - acc: 0.6044
Epoch 9/10
 - 0s - loss: 0.

Using average instead of sum prevents overfitting without the need of dropout layers

In [0]:
for i in range(4):
  model = Sequential()
  model.add(Embedding(vocab_size, 100, input_length=max_length))
  model.add(Dropout(0.5))
  model.add(Lambda(lambda x: mean(x, axis=1)))
  #model.add(Flatten())
  model.add(Dense(5, activation='linear'))
  print(model.summary())
  # compile network
  from keras import optimizers


  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(Xtrain, Ytrain, epochs=5, verbose=2)
  predicts=model.predict(Xtest)
  # evaluate
  loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
  print('Test Accuracy: %f' % (acc*100))
  precision,recall,f1 = calculateMetrics(predicts,Ytest,len(dictLabel))
  print("Precision = ",precision,"\n Recall = ",recall,"\nF1 = ",f1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (None, 66, 100)           332300    
_________________________________________________________________
dropout_9 (Dropout)          (None, 66, 100)           0         
_________________________________________________________________
lambda_32 (Lambda)           (None, 100)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 5)                 505       
Total params: 332,805
Trainable params: 332,805
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
 - 2s - loss: 2.4879 - acc: 0.1430
Epoch 2/5
 - 0s - loss: 1.6094 - acc: 0.1585
Epoch 3/5
 - 0s - loss: 1.6077 - acc: 0.2564
Epoch 4/5
 - 0s - loss: 1.6181 - acc: 0.2216
Epoch 5/5
 - 0s - loss: 1.6094 - acc: 0.0554
Test Accuracy: 5.791506
0
Precision = 

Using as an activation function the linear function we get a big range of different scores from 5% up to 12% thats is the score that we actually achieve

In [0]:

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Lambda(lambda x: mean(x, axis=1)))
model.add(Dense(5, activation='softmax'))
print(model.summary())


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
predicts=model.predict(Xtest)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
precision,recall,f1 = calculateMetrics(predicts,Ytest,len(dictLabel))
print("Precision = ",precision,"\n Recall = ",recall,"\nF1 = ",f1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_37 (Embedding)     (None, 66, 100)           332300    
_________________________________________________________________
lambda_37 (Lambda)           (None, 100)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 5)                 505       
Total params: 332,805
Trainable params: 332,805
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
 - 2s - loss: 1.5235 - acc: 0.4923
Epoch 2/10
 - 0s - loss: 1.3466 - acc: 0.6044
Epoch 3/10
 - 0s - loss: 1.1794 - acc: 0.6044
Epoch 4/10
 - 0s - loss: 1.0654 - acc: 0.6044
Epoch 5/10
 - 0s - loss: 1.0176 - acc: 0.6044
Epoch 6/10
 - 0s - loss: 0.9992 - acc: 0.6044
Epoch 7/10
 - 0s - loss: 0.9864 - acc: 0.6044
Epoch 8/10
 - 0s - loss: 0.9734 - acc: 0.6044
Epoch 9/10
 - 0s - loss: 0.

Using the softmax activation function we get similar results as the sigmoid activation function

In [0]:
print(max_length)
# define model
model = Sequential()
model.add(firstModel.wv.get_keras_embedding(train_embeddings=True))
model.add(Lambda(lambda x: mean(x, axis=1)))
model.add(Dense(5, activation='sigmoid'))
print("\nModel Summary:\n", model.summary())

           
# compile network
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
precision,recall,f1 = calculateMetrics(predicts,Ytest,len(dictLabel))
print("Precision = ",precision,"\n Recall = ",recall,"\nF1 = ",f1)

66
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_39 (Embedding)     (None, None, 100)         863200    
_________________________________________________________________
lambda_39 (Lambda)           (None, 100)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 5)                 505       
Total params: 863,705
Trainable params: 863,705
Non-trainable params: 0
_________________________________________________________________

Model Summary:
 None
Epoch 1/10
 - 2s - loss: 1.7732 - acc: 0.6031
Epoch 2/10
 - 0s - loss: 1.1071 - acc: 0.6031
Epoch 3/10
 - 0s - loss: 1.0321 - acc: 0.6031
Epoch 4/10
 - 0s - loss: 1.0298 - acc: 0.6031
Epoch 5/10
 - 0s - loss: 1.0265 - acc: 0.6031
Epoch 6/10
 - 0s - loss: 1.0234 - acc: 0.6031
Epoch 7/10
 - 0s - loss: 1.0192 - acc: 0.6044
Epoch 8/10
 - 0s - loss: 1.0163 - acc: 0.6044
Epoch 9

In [0]:
# define model
model = Sequential()
model.add(firstModel.wv.get_keras_embedding(train_embeddings=True))
model.add(Lambda(lambda x: sum(x, axis=1)))
model.add(Dense(5, activation='sigmoid'))
print("\nModel Summary:\n", model.summary())

           
# compile network
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
precision,recall,f1 = calculateMetrics(predicts,Ytest,len(dictLabel))
print("Precision = ",precision,"\n Recall = ",recall,"\nF1 = ",f1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_42 (Embedding)     (None, None, 100)         863200    
_________________________________________________________________
lambda_42 (Lambda)           (None, 100)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 5)                 505       
Total params: 863,705
Trainable params: 863,705
Non-trainable params: 0
_________________________________________________________________

Model Summary:
 None
Epoch 1/10
 - 2s - loss: 9.6157 - acc: 0.0438
Epoch 2/10
 - 0s - loss: 1.6281 - acc: 0.0631
Epoch 3/10
 - 0s - loss: 1.6281 - acc: 0.0657
Epoch 4/10
 - 0s - loss: 1.6281 - acc: 0.0606
Epoch 5/10
 - 0s - loss: 1.6121 - acc: 0.0554
Epoch 6/10
 - 0s - loss: 1.6094 - acc: 0.0554
Epoch 7/10
 - 0s - loss: 1.6094 - acc: 0.0554
Epoch 8/10
 - 0s - loss: 1.6094 - acc: 0.0554
Epoch 9/10

In [0]:
for i in range(4):
# define model
  model = Sequential()
  model.add(firstModel.wv.get_keras_embedding(train_embeddings=False))
  model.add(Lambda(lambda x: mean(x, axis=1)))
  model.add(Dense(5, activation='linear'))
  print("\nModel Summary:\n", model.summary())


  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(Xtrain, Ytrain, epochs=5, verbose=2)
  # evaluate
  loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
  print('Test Accuracy: %f' % (acc*100))
  precision,recall,f1 = calculateMetrics(predicts,Ytest,len(dictLabel))
  print("Precision = ",precision,"\n Recall = ",recall,"\nF1 = ",f1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_44 (Embedding)     (None, None, 100)         863200    
_________________________________________________________________
lambda_43 (Lambda)           (None, 100)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 5)                 505       
Total params: 863,705
Trainable params: 505
Non-trainable params: 863,200
_________________________________________________________________

Model Summary:
 None
Epoch 1/5
 - 2s - loss: 1.5627 - acc: 0.0606
Epoch 2/5
 - 0s - loss: 1.5437 - acc: 0.0619
Epoch 3/5
 - 0s - loss: 1.5517 - acc: 0.0619
Epoch 4/5
 - 0s - loss: 1.5487 - acc: 0.0619
Epoch 5/5
 - 0s - loss: 1.5448 - acc: 0.0619
Test Accuracy: 6.177606
0
Precision =  0.12046332046332046 
 Recall =  0.12046332046332046 
F1 =  0.12046332046332046
_________________________________

Using the linear activation function for the pretrained Embedding gives as consistently f1 score equals to 12%.

In [0]:


# define model
model = Sequential()
model.add(firstModel.wv.get_keras_embedding(train_embeddings=True))
model.add(Lambda(lambda x: mean(x, axis=1)))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(5, activation='sigmoid'))
print("\nModel Summary:\n", model.summary())

           
# compile network
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
precision,recall,f1 = calculateMetrics(predicts,Ytest,len(dictLabel))
print("Precision = ",precision,"\n Recall = ",recall,"\nF1 = ",f1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_49 (Embedding)     (None, None, 100)         863200    
_________________________________________________________________
lambda_48 (Lambda)           (None, 100)               0         
_________________________________________________________________
dense_51 (Dense)             (None, 32)                3232      
_________________________________________________________________
dense_52 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_53 (Dense)             (None, 5)                 85        
Total params: 867,045
Trainable params: 867,045
Non-trainable params: 0
_________________________________________________________________

Model Summary:
 None
Epoch 1/10
 - 3s - loss: 1.5336 - acc: 0.4884
Epoch 2/10
 - 0s - loss: 1.3607 - acc: 0.6031
Epoch 3/10
 - 0s - los

Adding more dence layers still doesn't improve the f1 score

In [0]:
# define model
model = Sequential()
model.add(firstModel.wv.get_keras_embedding(train_embeddings=False))
model.add(Lambda(lambda x: mean(x, axis=1)))
model.add(Dense(5, activation='sigmoid'))
print("\nModel Summary:\n", model.summary())

           
# compile network
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
precision,recall,f1 = calculateMetrics(predicts,Ytest,len(dictLabel))
print("Precision = ",precision,"\n Recall = ",recall,"\nF1 = ",f1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_50 (Embedding)     (None, None, 100)         863200    
_________________________________________________________________
lambda_49 (Lambda)           (None, 100)               0         
_________________________________________________________________
dense_54 (Dense)             (None, 5)                 505       
Total params: 863,705
Trainable params: 505
Non-trainable params: 863,200
_________________________________________________________________

Model Summary:
 None
Epoch 1/10
 - 2s - loss: 1.3478 - acc: 0.3222
Epoch 2/10
 - 0s - loss: 1.1350 - acc: 0.6031
Epoch 3/10
 - 0s - loss: 1.0370 - acc: 0.6031
Epoch 4/10
 - 0s - loss: 1.0310 - acc: 0.6031
Epoch 5/10
 - 0s - loss: 1.0298 - acc: 0.6031
Epoch 6/10
 - 0s - loss: 1.0274 - acc: 0.6031
Epoch 7/10
 - 0s - loss: 1.0276 - acc: 0.6044
Epoch 8/10
 - 0s - loss: 1.0272 - acc: 0.6044
Epoch 9/

By setting the trainable parametre False we get the same f1 score with much faster execution time

In [0]:
# define model
model = Sequential()
model.add(firstModel.wv.get_keras_embedding(train_embeddings=True))
model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_first'))
model.add(Lambda(lambda x: mean(x, axis=1)))
model.add(Dense(5, activation='sigmoid'))
print("\nModel Summary:\n", model.summary())

           
# compile network
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, Ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, Ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))
precision,recall,f1 = calculateMetrics(predicts,Ytest,len(dictLabel))
print("Precision = ",precision,"\n Recall = ",recall,"\nF1 = ",f1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_54 (Embedding)     (None, None, 100)         863200    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, None, 50)          0         
_________________________________________________________________
lambda_53 (Lambda)           (None, 50)                0         
_________________________________________________________________
dense_57 (Dense)             (None, 5)                 255       
Total params: 863,455
Trainable params: 863,455
Non-trainable params: 0
_________________________________________________________________

Model Summary:
 None
Epoch 1/10
 - 3s - loss: 1.5185 - acc: 0.5979
Epoch 2/10
 - 1s - loss: 1.1157 - acc: 0.6044
Epoch 3/10
 - 1s - loss: 1.0318 - acc: 0.6044
Epoch 4/10
 - 1s - loss: 1.0220 - acc: 0.6044
Epoch 5/10
 - 1s - loss: 1.0199 - acc: 0.6044
Epoch 6/10
 - 1s

Pooling doesn't improve the results either.

General observations

For most of our experiments the results were consistant and the f1 score never surpassed the 12%. Only exception was the linear activation function. The amount of data was not big enough to be able to increase f1 score.